## Lab 8: Text generation with GPT

During this lab, we will further explore the transformer architecture and GPT. The GPT (Generative Pre-trained Transformer) architecture has significantly advanced the field of NLP by enabling the development of powerful and versatile language models. Its transformer-based design, coupled with unsupervised pre-training on large text corpora, has revolutionized tasks such as text generation, summarization, and language understanding.

Even though we are not able to perform a large-scale training in the scope of this lab, we can still explore the capabilities of the model on a smaller scale by training on the `tiny_shakespeare` dataset and utilizing some pre-trained weights.


In [1]:
!pip install torch numpy transformers datasets tiktoken tqdm nltk bert_score torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 14.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_6

In [3]:
import os
import requests
import tiktoken
import numpy as np
import pickle
import torch
import time
import math
from contextlib import nullcontext

# don't forget to upload model.py into /content
from model import GPTConfig, GPT

### Data Preparation

Download the `tiny_shakespeare` dataset, which consists of numerous Shakespeare plays concatenated into a single text file.

It is encoded with Byte-Pair Encoding (BPE) that builds a vocabulary of subword units to optimally represent the input data. The encoded tokens for each split (train/val) are saved into corresponding binary files.

In [ ]:
input_file_path = os.path.join(os.path.abspath(''), 'input.txt')
if not os.path.exists(input_file_path):
    data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    with open(input_file_path, 'w', encoding='utf-8') as f:
        f.write(requests.get(data_url).text)

with open(input_file_path, 'r', encoding='utf-8') as f:
    data = f.read()

n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# encode with tiktoken gpt2 bpe
enc = tiktoken.get_encoding("gpt2")
train_ids = enc.encode_ordinary(train_data)
val_ids = enc.encode_ordinary(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(os.path.join(os.path.abspath(''), 'train.bin'))
val_ids.tofile(os.path.join(os.path.abspath(''), 'val.bin'))

### Train a small GPT model from scratch

Let's train the model for the task of next token prediction

In [ ]:
# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O

init_from = 'scratch'  # 'scratch' or 'resume' or 'gpt2*'
out_dir = 'out-shakespeare'
eval_interval = 100
eval_iters = 100
log_interval = 20
always_save_checkpoint = True  # if True, always save a checkpoint after each eval

# data
dataset = 'shakespeare'
gradient_accumulation_steps = 5 * 8  # used to simulate larger batch sizes
batch_size = 12  # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 128

# model
# ------------------------------------------------------------------------------
# play with these parameters! if you have access to the GPU runtime, make the model
# bigger, it has a significant influence on its performance
n_layer = 6
n_head = 4
n_embd = 128
# ------------------------------------------------------------------------------
dropout = 0.0  # for pretraining 0 is good, for finetuning try 0.1+
bias = False  # do we use bias inside LayerNorm and Linear layers?

# adamw optimizer
learning_rate = 6e-4  # max learning rate
max_iters = 500  # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0  # clip gradients at this value, or disable if == 0.0

# learning rate decay settings
decay_lr = True  # whether to decay the learning rate
warmup_iters = 10  # how many steps to warm up for
lr_decay_iters = 500  # should be ~= max_iters per Chinchilla
min_lr = 6e-5  # minimum learning rate, should be ~= learning_rate/10 per Chinchilla

# system
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'  # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True  # use PyTorch 2.0 to compile the model to be faster
print(f'[.] {device} chosen as device')

# -----------------------------------------------------------------------------
config_keys = [k for k, v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys}
# -----------------------------------------------------------------------------


# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, block_size=block_size)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters

    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr

    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # coeff ranges 0..1

    return min_lr + coeff * (learning_rate - min_lr)


def get_batch(split, batch_size=16, block_size=1024):
    # We recreate np.memmap every batch to avoid a memory leak, as per
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == 'train':
        data = np.memmap(os.path.join(os.path.abspath(''), 'train.bin'),
                         dtype=np.uint16, mode='r')
    else:
        data = np.memmap(os.path.join(os.path.abspath(''), 'val.bin'),
                         dtype=np.uint16, mode='r')

    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i + block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i + 1:i + 1 + block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

Various inits, derived attributes, I/O setup

In [ ]:
seed_offset = 0
tokens_per_iter = gradient_accumulation_steps * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

os.makedirs(out_dir, exist_ok=True)

torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True  # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True  # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu'  # for later use in torch.autocast

# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

data_dir = os.path.join('data', dataset)

# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# attempt to derive vocab_size from the dataset
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout)

# init a new model from scratch
print("Initializing a new model from scratch")
# determine the vocab size we'll use for from-scratch training
if meta_vocab_size is None:
    print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")

model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
gptconf = GPTConfig(**model_args)
model = GPT(gptconf)

# crop down the model block size if desired, using model surgery
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size  # so that the checkpoint will have the right value

model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
if init_from == 'resume':
    optimizer.load_state_dict(checkpoint['optimizer'])
checkpoint = None  # free up memory

# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model)  # requires PyTorch 2.0

In [ ]:
# training loop
X, Y = get_batch('train', block_size=block_size)  # fetch the very first batch
t0 = time.time()
local_iter_num = 0  # number of iterations in the lifetime of this process

while True:
    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        with ctx:
            logits, loss = model(X, Y)
            loss = loss / gradient_accumulation_steps  # scale the loss to account for gradient accumulation
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = get_batch('train', block_size=block_size)
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()

    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()

    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt * 1000:.2f}ms")

    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

> In case you are having trouble with securing a GPU runtime and training the model, download the trained weights from [here](https://drive.google.com/file/d/17gfJ76SyGJVW3jinz3Xv5B7XxTyE9NNA/view?usp=sharing). Create the directory called `out-shakespeare` and place the downloaded weights there.

In [8]:
if not os.path.exists('/content/out-shakespeare'):
  os.makedirs('/content/out-shakespeare')

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=17gfJ76SyGJVW3jinz3Xv5B7XxTyE9NNA' -O /content/out-shakespeare/ckpt.pt

---
### Sample from a trained model

Initialize the trained model from a directory.

<div class="alert alert-block alert-info"><b>Tip:</b> There is no need to re-initialize the model and load it into memory again if you've just trained it. Run the next cell if the context of the notebook was reset after the training or if you downloaded the weights.</div>

In [10]:
# -----------------------------------------------------------------------------
init_from = 'resume'
out_dir = 'out-shakespeare' # ignored if init_from is not 'resume'

seed = 1337
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = True # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


# init from a model saved in a specific directory
ckpt_path = os.path.join(out_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)

model.load_state_dict(state_dict)
model.eval()
model.to(device)

if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

number of parameters: 7.62M


In [11]:
# assume gpt-2 encodings by default
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

---
### Sample from the trained model

We can prompt the model by providing a context. Try sampling with a different context and sample length

In [ ]:
# encode the beginning of the prompt
context = 'The Universe is vast'
start_ids = encode(context)
num_samples = 5
sample_len = 128

temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability


with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

            probs, y = model.generate(x, sample_len, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('==============================')

The generated text mostly does not make sense but the model could definitely capture some attributes of the Shakespearean style.

Let's run the model on the validation dataset. Experiment with the context size and see how it influences the generation.

> Note: the context, provided to the model, consists of the first few tokens from each sample. Be shure to exclude is from evaluation as it will always be "right" and skew the metrics

In [ ]:
# experiment with sample length, context size and their influence on the evaluation metrics
sample_len = 64
batch_size = 32
start_len = 5

temperature = 0.8
top_k = 200
# -------------------------------------

val_data = np.memmap('./val.bin', dtype=np.uint16, mode='r')
num_batches = len(val_data) // sample_len // batch_size


with torch.no_grad():
    with ctx:
        for batch_i in range(num_batches):
            print(f'batch {batch_i}/{num_batches}')

            X_val, _ = get_batch('val', batch_size, sample_len)

            for k in range(batch_size):
                start_ids = X_val[k, :start_len]
                # print('START:', start_ids, ' - "', decode(list(start_ids)), '"')
                x = start_ids.clone().detach().type(torch.long).to(device)[None, ...]

                probs, pred = model.generate(x, sample_len-start_len,
                                             temperature=temperature, top_k=top_k)

                decoded_pred = decode(pred[0].tolist())
                decoded_gt = decode(X_val[k].tolist())

                # print('PRED DECODED: ', decoded_pred)
                # print('--------------------------')
                # print('GT DECODED: ', decoded_gt)
                # print('==============================')

### Model evaluation

It is important to be able to quantitatively evaluate language models. Some of the popular evaluation metrics that use reference text are BLEU score, BERTScore and Perplexity.

<div class="alert alert-block alert-info"><b>Tip:</b> Save the needed predictions while running the model on the validation dataset in the cell above. Computing the metrics on the dataset level is more straightforward.</div>

#### 1. BLEU score

The BLEU (Bilingual Evaluation Understudy) score works by comparing the n-grams (contiguous sequences of n tokens) in the generated text to those in the reference text(s). It calculates a precision score for each n-gram size (typically up to 4-grams) and combines these scores using a weighted geometric mean.

<div class="alert alert-block alert-warning"><b>Challenge 1:</b> Use the NLTK framework to calculate BLEU scores for 1-, 2-, 3-, and 4-grams on the validation dataset.</div>

In [ ]:
...

print('BLEU-1: ', bleu1)
print('BLEU-2: ', bleu2)
print('BLEU-3: ', bleu3)
print('BLEU-4: ', bleu4)

#### 2. BERTScore

BERTScore  leverages contextual embeddings from BERT (Bidirectional Encoder Representations from Transformers) to compute similarity between sentences or text spans. Unlike BLEU score that works on a token level, it considers both word overlap and contextual information, providing a more accurate evaluation.

Compared to BLEU score, BERTScore offers several advantages:

- Contextual understanding: BERTScore considers the contextual meaning of words, capturing nuances that BLEU, which relies solely on word overlap, may miss.
- Robustness to word order: BERTScore's contextual embeddings enable it to handle variations in word order, making it more robust to changes in sentence structure or word arrangement.
- Higher correlation with human judgment: BERTScore has been shown to correlate better with human judgment in evaluating text quality, especially in tasks like summarization and text generation.

<div class="alert alert-block alert-warning"><b>Challenge 2:</b> Use the bert_score package to calculate BERTScore on the validation dataset.</div>

In [ ]:
...

print(f"BERTScore Precision: {bs_precision:.4f}, Recall: {bs_recall:.4f}, F1: {bs_f1:.4f}")

#### 3. Perplexity

Perplexity is a measurement used in natural language processing (NLP) to assess how well a language model predicts a sample of text. It quantifies the average uncertainty or surprise of the model in predicting the next word or token in a sequence. Lower perplexity values indicate that the model is more confident and accurate in its predictions, while higher values suggest more uncertainty.

<div class="alert alert-block alert-warning"><b>Challenge 3:</b> Use the torcheval package to calculate perplexity on the validation dataset.</div>

In [ ]:
...

print('Perplexity: ', perplexity)

---
Now let's load a pre-trained GPT-2 model and see, how does it perform in terms of the calculated metrics

In [ ]:
init_from = 'gpt2-medium'  # 'gpt2-xl' if you have access to a decent GPU

# init from a given GPT-2 model
model = GPT.from_pretrained(init_from, dict(dropout=0.0))
model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

Sampling the model with a given context

In [ ]:
# encode the beginning of the prompt
context = 'The Universe is vast'
start_ids = encode(context)
num_samples = 5
sample_len = 128

with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

            probs, y = model.generate(x, sample_len, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('==============================')

<div class="alert alert-block alert-warning"><b>Challenge 4:</b> Re-use the code from above to run the model on the validation dataset and calculate BLEU score, BERTScore and perplexity. What do you observe? Do numbers correlate with the qualitative evaluation?</div>

In [ ]:
...

In [ ]:
...

print('BLEU-1: ', bleu1)
print('BLEU-2: ', bleu2)
print('BLEU-3: ', bleu3)
print('BLEU-4: ', bleu4)

In [ ]:
...

print(f"BERTScore Precision: {bs_precision:.4f}, Recall: {bs_recall:.4f}, F1: {bs_f1:.4f}")

In [ ]:
...

print('Perplexity: ', perplexity)

> Do you see drawbacks of the metrics that rely on the reference text? Can we provide an adequate reference in case of an unconstrained text generation? Compare the outputs of both models qualitatively. Think about the other ways to evaluate text generation models.